In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics import ndcg_score, mean_squared_error

# 1. Load Data (Same Split)
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 2. Compute "The Model" (Just a dictionary of averages)
# User Mean: "How generous is this user?"
user_means = train_df.groupby('User_ID')['Book_Rating'].mean().to_dict()

# Global Mean: "What is the average rating generally?" (Fallback)
global_mean = train_df['Book_Rating'].mean()

# Popularity List (Needed for ranking because User Mean gives ties)
# We calculate the "Top Books" to recommend when we can't distinguish items
popular_books = train_df.groupby('Book_Title')['Book_Rating'].count() \
    .sort_values(ascending=False).index.tolist()

# 3. Create Helpers for Scorecard
# (Same setup as NCF/Cosine to ensure fair comparison)
df_full = pd.concat([train_df, test_df])

# We need encoders just to build the matrix for the Referee
# (The baseline model doesn't use them, but the Scorecard needs them)
from sklearn.preprocessing import LabelEncoder
user_enc = LabelEncoder()
df_full['user_encoded'] = user_enc.fit_transform(df_full['User_ID'])
book_enc = LabelEncoder()
df_full['book_encoded'] = book_enc.fit_transform(df_full['Book_Title'])

# Split back to get the Test Matrix
test_df_encoded = df_full.iloc[len(train_df):]
n_users = df_full['user_encoded'].max() + 1
n_books = df_full['book_encoded'].max() + 1

test_m = csr_matrix(
    (test_df_encoded['Book_Rating'].values, 
     (test_df_encoded['user_encoded'].values, test_df_encoded['book_encoded'].values)), 
    shape=(n_users, n_books)
)

# Popularity Dict for Novelty Score
book_pop_dict = df_full.groupby('book_encoded')['Book_Rating'].count().to_dict()

print(f"Baseline Trained. Global Mean: {global_mean:.2f}")

Baseline Trained. Global Mean: 7.82


In [6]:
def predict_rating_user_avg(user_id, book_title):
    # 1. Check if we know this user
    if user_id in user_means:
        return user_means[user_id]
    
    # 2. Cold Start: Return Global Average
    return global_mean

# Test
# If User 276747 usually gives 8s, this should print 8.0
print(f"Predicted Score: {predict_rating_user_avg(276747, 'Any Book')}")

Predicted Score: 7.8193204757156405


In [7]:
def recommend_user_avg(user_id, n_recommendations=5):
    # 1. Get the score we will predict for EVERYTHING
    predicted_score = user_means.get(user_id, global_mean)
    
    # 2. Get Top Popular Books (that user hasn't read)
    # (Simplified: Just taking top N popular for the baseline demo)
    recs = popular_books[:n_recommendations]
    
    print(f"--- Baseline Recommendations for User {user_id} ---")
    results = []
    for title in recs:
        # The score is constant!
        print(f"{predicted_score:.2f} | {title}")
        results.append(title)
        
    return results

# Test
recs = recommend_user_avg(276747)

--- Baseline Recommendations for User 276747 ---
7.82 | The Lovely Bones: A Novel
7.82 | The Da Vinci Code
7.82 | The Secret Life of Bees
7.82 | Bridget Jones's Diary
7.82 | Wild Animus


In [8]:
import numpy as np
from sklearn.metrics import ndcg_score, mean_squared_error

# 1. Create Helper Mapping (Required for the wrapper)
# Maps the Integers (0, 1, 2...) back to Original User IDs (276747...)
int_to_user_id = dict(zip(df_full['user_encoded'], df_full['User_ID']))

# 2. Define the Wrapper Function
def predict_baseline_wrapper(u_enc, b_enc):
    # Convert Encoded Integer -> Original User ID
    user_id = int_to_user_id.get(u_enc)
    
    # Predict (Constant score for this user)
    if user_id in user_means:
        return user_means[user_id]
    return global_mean

# 3. Define the Scorecard Function (With Novelty included)
def get_model_scorecard(model_name, test_data_matrix, prediction_function, book_popularity_dict):
    print(f"--- Scoring Model: {model_name} ---")
    rmses = []
    ndcg_scores = []
    novelty_scores = []
    
    test_users = np.unique(test_data_matrix.nonzero()[0])
    sample_users = np.random.choice(test_users, size=min(500, len(test_users)), replace=False)
    
    for u in sample_users:
        true_ratings = test_data_matrix[u].data
        if len(true_ratings) < 2: continue 
        
        # Predict
        true_book_ids = test_data_matrix[u].indices
        pred_ratings = [prediction_function(u, b) for b in true_book_ids]
            
        # Metrics
        rmses.append(np.sqrt(mean_squared_error(true_ratings, pred_ratings)))
        try:
            ndcg_scores.append(ndcg_score([true_ratings], [pred_ratings]))
        except: pass
        
        # Novelty
        top_k_idx = np.argsort(pred_ratings)[::-1][:5]
        top_books = true_book_ids[top_k_idx]
        pop_score = np.mean([book_popularity_dict.get(b, 0) for b in top_books])
        novelty_scores.append(pop_score)

    return {
        "Model": model_name,
        "RMSE": np.mean(rmses),
        "NDCG": np.mean(ndcg_scores),
        "Novelty": np.mean(novelty_scores)
    }

# 4. Run Evaluation
baseline_scores = get_model_scorecard("User Average Baseline", test_m, predict_baseline_wrapper, book_pop_dict)
print(baseline_scores)

--- Scoring Model: User Average Baseline ---
{'Model': 'User Average Baseline', 'RMSE': np.float64(1.5308406216825525), 'NDCG': np.float64(0.9618240823162386), 'Novelty': np.float64(55.28770287141073)}
